## 01/ Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Enable inline plotting in the notebook
%matplotlib inline

In [ ]:
# Read the XLSX file
data_2010 = pd.read_excel("../data/bronze.xlsx",sheet_name="Year 2009-2010")
data_2011 = pd.read_excel("../data/bronze.xlsx",sheet_name="Year 2010-2011")

In [ ]:
datas = [data_2010, data_2011]
data = pd.concat(datas, keys = ['09-10', '10-11'])
data

### Attribute Information:

- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.

- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.

- Description: Product (item) name. Nominal.

- Quantity: The quantities of each product (item) per transaction. Numeric.

- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.

- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).

- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.

- Country: Country name. Nominal. The name of the country where a customer resides.

In [ ]:
from ydata_profiling import ProfileReport

# Generate the profiling report
profile = ProfileReport(data, title="Profiling Report")

# Save the report as an HTML file
profile.to_file("profiling_report.html")

In [ ]:
data.rename(index = str, \
              columns = {'Invoice' : 'invoice', \
                         'StockCode' : 'stock_code', \
                         'Quantity' : 'quantity', \
                         'InvoiceDate' : 'invoice_date', \
                         'Price' : 'unit_price', \
                         'Country' : 'country', \
                         'Description' : 'description', \
                         'Customer ID' : 'customer_id'}, inplace = True)
data

In [ ]:
data.info()

In [ ]:
# number of unique customers
data["customer_id"].nunique()

In [ ]:
# Check the statistical information for data for null values:
data.isnull().sum().sort_values(ascending = False)

In [ ]:
data.describe()

In [ ]:
# Calculate how many negative values of unit_price are present in retail. If they are only a small percentage of the total number of instances, then remove them from retail:
(data['unit_price'] <= 0).sum()

In [ ]:
# Similarly, find out the number of negative values of quantity:
(data['quantity'] <= 0).sum()

In [ ]:
# Display the total number of negative instances for the unit_price and quantity columns:
data[(data['unit_price'] <= 0) & (data['quantity'] <= 0) \
       & (data['customer_id'].isnull())]

In [ ]:
data[(data['quantity'] <= 0) ]

In [ ]:
data[(data['unit_price'] < 0) ]
# Adjust bad debt = Ajustement des créances douteuses

In [ ]:
# Store the instances with missing values in another DataFrame:
null_data = data[data.isnull().any(axis=1)]
null_data

In [ ]:
# # Delete the instances with missing values from retail:
# new_data = data[(data['unit_price'] > 0) \
#                     & (data['quantity'] > 0)]
# new_data.describe()

In [ ]:
# Drop rows with NaN values in the customer_id column
del_data = data.dropna(subset=['customer_id', 'description'])

del_data

In [ ]:
del_data.info()

In [ ]:
# del_data.plot()

In [ ]:
# Plot a boxplot for unit_price to see if there are any outliers. If there are, remove those instances:
plt.subplots(figsize = (12, 6))
sns.boxplot(del_data.unit_price)

In [ ]:
del_data.drop_duplicates(inplace=True)
df = del_data
df

In [ ]:
# Plot a boxplot for unit_price to see if there are any outliers. If there are, remove those instances:
plt.subplots(figsize = (12, 6))
sns.boxplot(df.unit_price)

In [ ]:
# Filter the negative values
negative_values = del_data.unit_price[del_data.unit_price < 0]
negative_values

In [ ]:
df = df[(df['unit_price'] > 0) & (df['unit_price'] < 15000)]
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sort_values(ascending = False)

In [ ]:

# Create a figure and axes
fig, ax = plt.subplots()

# Plot the boxplot
ax.boxplot(df.unit_price)

# Set title

ax.set_title('Price')

# Display the plot
plt.show()

In [ ]:
# number of unique customers
df["customer_id"].nunique()

In [ ]:
# change type of customer_id from float to string
data['customer_id']=data['customer_id'].astype(str)
data.info()

### Remarks:
- 

## 02/ Data Analysis

### Customer Segmentation with RFM

In [ ]:
import datetime as dt

In [ ]:
# Add a new column which calculates the total despense
df['TotalPrice'] = df['quantity'] * df['unit_price']
df.head(2)

### Calculating RFM Metrics (Recency, frequency and monetary)
- R for how recently did the customer purchase? This information can be used to remind recent customers to revisit the business soon to continue meeting their purchase needs.

- F for how often do the customer purchase? Predicting this can assist marketing efforts directed at reminding the customer to visit the business again.

- M for how much do the customer spend? While this can produce a better return on investment in marketing and customer service, it also runs the risk of alienating customers who have been consistent but have not spent as much with each transaction.

### RFM Segments
- Champions: Your best customers, they buy and spend a lot and made their last purchase recently.

- Loyal Customer: Very good customers. They spend a lot.

- Potential Loyalist: Recent customers, but who have already spent a lot.

- New Customer: Recent customers, who made only a few purchases.

- Promising: Customers who buy frequently and spend a lot, but made their last purchase some time ago.

- Need Attention: Customers with recency and above-average spending.

- At Risk: Customers who bought frequently, but haven't made any purchases in a long time.

- Can't lose them: Customers who have spent a lot, but have been inactive for a long time.

- Hibernate: Low-frequency, low-spender customers who haven't bought in a long time.

- Lost: Your worst customers. They haven't bought in a long time, they only bought once and they spent very little.

#############################################################"

In [ ]:
# Définir une fonction de conversion pour la colonne datetime
def parse_datetime(date_string):
    return pd.to_datetime(date_string)

# Spécifier les types de données et les convertisseurs
dtype = {'invoice': str, 'object': int, 'stock_code': object, 'description': object, 'quantity': int, 'unit_price': float, 'customer_id': object, 'country': object}
converters = {'invoice_date': parse_datetime}

# Charger le fichier CSV en utilisant les types de données et les convertisseurs
data = pd.read_csv("../data/silver.csv", dtype=dtype, converters=converters)

# Vérifier les types de données
print(data.dtypes)

In [ ]:
# Spécifier les types de données et les convertisseurs
dtype = {'invoice_date': int,'invoice': int, 'total_price': float}

# Charger le fichier CSV en utilisant les types de données et les convertisseurs
rfm = pd.read_csv("../data/rfm_data.csv", dtype=dtype)

# Vérifier les types de données
print(rfm.dtypes)

In [ ]:
rfm

In [ ]:
rfm.rename(columns={'invoice_date': 'Recency', 'invoice': 'Frequency', 'total_price': 'Monetary'}, inplace=True)
rfm

In [ ]:
rfm.describe()

In [ ]:
sns.scatterplot(x=rfm['Frequency'], y=rfm['Monetary'])
plt.xlim([0,150])
plt.ylim([0,10000])
plt.show()

In [ ]:
sns.boxplot(rfm['Frequency'])

In [ ]:
sns.boxplot(rfm.Monetary)

In [ ]:
sns.boxplot(rfm.Recency)

In [ ]:
# 3D scatterplot of Recency, Frequency and Monetary
plt.figure(dpi=150)
ax = plt.axes(projection='3d')
ax.scatter3D(rfm.Recency, rfm.Frequency, rfm.Monetary)
# ax.view_init(30,30)

In [ ]:
# Remove outliers using z-score
from scipy import stats
c = ['Recency', 'Frequency', 'Monetary']
# rfm.shape
rfmno = rfm[c][(np.abs(stats.zscore(rfm[c])) < 3).all(axis=1)]

# 3D scatterplot of Recency, Frequency and Monetary
plt.figure(dpi=150)
ax = plt.axes(projection='3d')
ax.scatter3D(rfmno.Recency, rfmno.Frequency, rfmno.Monetary)
# ax.view_init(30,30)



In [ ]:
rfm

In [ ]:
from sklearn.preprocessing import StandardScaler
rfms = StandardScaler().fit(rfm[c]).transform(rfm[c])
rfms = pd.DataFrame(rfms, columns=c)
rfmsno = rfms[c][np.abs(rfms[c] < 3).all(axis=1)]
rfmsno

In [ ]:
from sklearn.cluster import KMeans

plt.figure(dpi=100)
inertias = []
for i in range(1,15):
    inertias.append(KMeans(n_clusters = i).fit(rfm[c]).inertia_)
plt.plot(range(1,15), inertias)
plt.ylabel('Inertia')
plt.xlabel('Number of Clusters')
plt.title('KMeans Performance By Cluster Number')



In [ ]:
kp = pd.Series(KMeans(n_clusters = 3, random_state=10, max_iter=3000).fit(rfms[c]).predict(rfms[c]))
rfm['Cluster'] = kp

In [ ]:
rfm.Cluster.value_counts()

In [ ]:
plt.figure(dpi=150)
ax = plt.axes(projection='3d')
ax.scatter3D(rfm.Recency, rfm.Frequency, rfm.Monetary, c=rfm.Cluster.values, cmap='gist_rainbow')
plt.xlabel('Recency')
plt.ylabel('Frequency')
ax.set_zlabel('Monetary Value')
plt.title('Customer Clusters')
plt.legend()

In [ ]:
rfm[c + ['Cluster']].groupby('Cluster').mean()

In [ ]:
plt.scatter(rfm.Frequency, rfm.Monetary,c=rfm.Segment)